# Laboratorio 2: Armado de un esquema de aprendizaje automático

En el laboratorio final se espera que puedan poner en práctica los conocimientos adquiridos en el curso, trabajando con un conjunto de datos de clasificación.

El objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

El conjunto de datos a utilizar está en `./data/loan_data.csv`. Si abren el archivo verán que al principio (las líneas que empiezan con `#`) describen el conjunto de datos y sus atributos (incluyendo el atributo de etiqueta o clase).

Se espera que hagan uso de las herramientas vistas en el curso. Se espera que hagan uso especialmente de las herramientas brindadas por `scikit-learn`.

In [35]:
import numpy as np
import pandas as pd

# TODO: Agregar las librerías que hagan falta
from sklearn.model_selection import train_test_split

## Carga de datos y división en entrenamiento y evaluación

La celda siguiente se encarga de la carga de datos (haciendo uso de pandas). Estos serán los que se trabajarán en el resto del laboratorio.

In [36]:
#dataset = pd.read_csv("./data/loan_data.csv", comment="#")
dataset = pd.read_csv("https://raw.githubusercontent.com/DiploDatos/IntroduccionAprendizajeAutomatico/master/data/loan_data.csv", comment="#")


# División entre instancias y etiquetas
X, y = dataset.iloc[:, 1:], dataset.TARGET

# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Documentación:

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

## Preprocesamiento

In [37]:
# valores nulos
print(dataset.isnull().sum())

TARGET     0
LOAN       0
MORTDUE    0
VALUE      0
YOJ        0
DEROG      0
DELINQ     0
CLAGE      0
NINQ       0
CLNO       0
DEBTINC    0
dtype: int64


Verificamos que no hay valores nulos. 

In [38]:
# distribución de variable target en subconjuntos de train y test
print("Distribución original:", y.value_counts(normalize=True))
print("Train:", y_train.value_counts(normalize=True))
print("Test:", y_test.value_counts(normalize=True))


Distribución original: TARGET
0    0.833333
1    0.166667
Name: proportion, dtype: float64
Train: TARGET
0    0.836143
1    0.163857
Name: proportion, dtype: float64
Test: TARGET
0    0.822102
1    0.177898
Name: proportion, dtype: float64


Verificamos que los subconjuntos de train y test presentan distribuciones semejantes de la variable TARGET. 

## Ejercicio 1: Descripción de los Datos y la Tarea

Responder las siguientes preguntas:

1. ¿De qué se trata el conjunto de datos?

The Home Equity dataset (HMEQ) contains baseline and loan performance
information for 5,960 recent home equity loans.


2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?

The target (BAD) is a binary
variable indicating whether an applicant eventually defaulted or was
seriously delinquent. 
This adverse outcome occurred in 1,189 cases (20%). 

3. ¿Qué información (atributos) hay disponible para hacer la predicción?

For
each applicant, 12 input variables were recorded. 

LOAN    Amount of the loan request
MORTDUE Amount due on existing mortgage
VALUE   Value of current property
YOJ     Years at present job
DEROG   Number of major derogatory reports
DELINQ  Number of delinquent credit lines
CLAGE   Age of oldest trade line in months
NINQ    Number of recent credit lines
CLNO    Number of credit lines
DEBTINC Debt-to-income ratio

| Variable           | Tipo              | Descripción                                                                                                                                                |
| ------------------ | ----------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **TARGET** (o BAD) | Binaria (0/1)     | **Variable objetivo.** Indica si el solicitante cayó en default:<br>🟢 `0` = Préstamo pagado correctamente.<br>🔴 `1` = Incumplimiento o morosidad severa. |
| **LOAN**           | Numérica continua | Monto solicitado en el préstamo.                                                                                                                    |
| **MORTDUE**        | Numérica continua | Saldo actual de la hipoteca existente sobre la propiedad.                                                                                                  |
| **VALUE**          | Numérica continua | Valor estimado de la propiedad del solicitante.                                                                                                            |
| **YOJ**            | Numérica discreta | Años de antigüedad del solicitante en su empleo actual.                                                                                                    |
| **DEROG**          | Numérica discreta | Cantidad de reportes importantes de crédito negativo (ej. bancarrotas, juicios).                                                                           |
| **DELINQ**         | Numérica discreta | Número de líneas de crédito con moras registradas.                                                                                                         |
| **CLAGE**          | Numérica continua | Antigüedad de la línea de crédito más antigua, en meses. Es un indicador de experiencia crediticia.                                                        |
| **NINQ**           | Numérica discreta | Número de líneas de crédito abiertas recientemente (indicador de actividad reciente).                                                               |
| **CLNO**           | Numérica discreta | Número total de líneas de crédito abiertas (tarjetas, préstamos, etc.).                                                                                    |
| **DEBTINC**        | Numérica continua | Relación deuda-ingresos (%). Se calcula como: *(total de deudas mensuales / ingreso mensual)*. Un valor alto puede indicar mayor riesgo crediticio.        |


4. ¿Qué atributos imagina ud. que son los más determinantes para la predicción?

- **DEBTINC**: Es un indicador directo de la capacidad de pago del solicitante. Una persona con mucha deuda en relación a sus ingresos tiene más riesgo de incumplir.

Cuanto más alto sea este ratio, más probable es que haya dificultad para cumplir con nuevas obligaciones.

- **DEROG**: La presencia de eventos crediticios graves (como bancarrota o juicios) es un fuerte indicador de riesgo histórico.

- **DELINQ**: Aunque menos grave que DEROG, refleja incumplimientos recientes o frecuentes, lo cual es útil para predecir problemas futuros.

**No hace falta escribir código para responder estas preguntas.**

## Ejercicio 2: Predicción con Modelos Lineales

En este ejercicio se entrenarán modelos lineales de clasificación para predecir la variable objetivo.

Para ello, deberán utilizar la clase SGDClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/sgd.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


In [39]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

### Escalado de features

Realizaremos un escaleo de los datos ya que SGDClasiffier usa el método de descenso por gradiente, el cual es sensible a la escala. 

In [40]:
# Todas las columnas son numéricas en este dataset
numeric_features = X.columns.tolist()

numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
])


In [41]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])


### Ejercicio 2.1: SGDClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador SGDClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

In [42]:
## valores de SGDClassifier por defecto
SGDClassifier??

Init signature:
SGDClassifier(
    loss='hinge',
    *,
    penalty='l2',
    alpha=0.0001,
    l1_ratio=0.15,
    fit_intercept=True,
    max_iter=1000,
    tol=0.001,
    shuffle=True,
    verbose=0,
    epsilon=0.1,
    n_jobs=None,
    random_state=None,
    learning_rate='optimal',
    eta0=0.0,
    power_t=0.5,
    early_stopping=False,
    validation_fraction=0.1,
    n_iter_no_change=5,
    class_weight=None,
    warm_start=False,
    average=False,
)
Source:        
class SGDClassifier(BaseSGDClassifier):
    """Linear classifiers (SVM, logistic regression, etc.) with SGD training.

    This estimator implements regularized linear models with stochastic
    gradient descent (SGD) learning: the gradient of the loss is estimated
    each sample at a time and the model is updated along the way with a
    decreasing strength schedule (aka learning rate). SGD allows minibatch
    (online/out-of-core) learning via the `partial_fit` method.
    For best results using the default lear

In [43]:
# ===============================
# Pipeline completo (prepro + modelo)
# ===============================
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(random_state=42))
])


#### Entrenamiento y predicción

In [44]:
# ===============================
# Entrenamiento
# ===============================
clf.fit(X_train, y_train)

# ===============================
# Predicción
# ===============================
y_pred = clf.predict(X_test)

#### Evaluación

In [45]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# ============
# TRAIN
# ============
y_train_pred = clf.predict(X_train)
print("🔹 Evaluación en conjunto de ENTRENAMIENTO")
print("Accuracy:", accuracy_score(y_train, y_train_pred))
print("Precision:", precision_score(y_train, y_train_pred))
print("Recall:", recall_score(y_train, y_train_pred))
print("F1 Score:", f1_score(y_train, y_train_pred))
print("Matriz de confusión:\n", confusion_matrix(y_train, y_train_pred))

# ============
# TEST
# ============
y_test_pred = clf.predict(X_test)
print("\n🔹 Evaluación en conjunto de TEST")
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("Precision:", precision_score(y_test, y_test_pred))
print("Recall:", recall_score(y_test, y_test_pred))
print("F1 Score:", f1_score(y_test, y_test_pred))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_test_pred))


🔹 Evaluación en conjunto de ENTRENAMIENTO
Accuracy: 0.8570465273095077
Precision: 0.6099290780141844
Recall: 0.35390946502057613
F1 Score: 0.4479166666666667
Matriz de confusión:
 [[1185   55]
 [ 157   86]]

🔹 Evaluación en conjunto de TEST
Accuracy: 0.8571428571428571
Precision: 0.6444444444444445
Recall: 0.4393939393939394
F1 Score: 0.5225225225225225
Matriz de confusión:
 [[289  16]
 [ 37  29]]


#### Interpretación

- Accuracy: proporción de predicciones correctas (puede ser engañoso en datasets con proporción de éxito/fracaso desbalanceada)

- Precision: de todos los positivos que predijo el modelo, ¿cuántos eran realmente positivos?

- Recall: de todos los positivos reales, ¿cuántos detectó el modelo?

- F1-score: media armónica entre precision y recall.

- Matriz de confusión: muestra TP, FP, TN, FN.

##### Matriz de confusión en contexto del problema
- FP: Se predijo mora cuando el solicitante pagó correctamente el crédito. 
- FN: Se predijo cumplimiento cuando el solicitante incurrió en mora. 
- TP: Se predijo mora y hubo mora. 
- TN: Se predijo cumplimiento y hubo cumplimiento. 

El modelo presenta un valor de **accuracy** de 0.82. Aunque a primera vista parece alto, hay que tener presente que el dataset presenta un 20% de casos de mora (`target = 1`), por lo que un modelo que predijera `target = 0` para todos los casos tendría una accuracy de alrededor de 0.80. 

El valor de **precisión** es menor al 50% en las evaluaciones de ambos subconjuntos de datos. Esto significa que de los casos en que el modelo predijo que iba a haber mora, menos de la mitad lo fueron. Este no es un resultado alentador para la implementación de este modelo ya que indica una alta proporción de falsos positivos. 

El valor de **recall**, menor al 30% en ambos subconjuntos de datos, indica que el modelo no está detectando más del 70% de los casos reales de mora. 

EL F1-score es el indicador más integral y confiable, sobre todo en un conjunto de datos con clases desbalanceadas. Valores tan bajos de F1-score reflejan un modelo con rendimiento pobre para detectar los casos de mora. 

#### Conclusión

El modelo es débil para tareas sensibles como evaluación crediticia. 

Se debe ajustar los hiperparámetros para mejorar la performance. 

### Ejercicio 2.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del SGDClassifier. Como mínimo, probar diferentes funciones de loss, tasas de entrenamiento y tasas de regularización.

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

El siguiente código explora:
- 3 funciones de pérdida
    - log-loss: regresión logística
    - hinge: SVM lineal
    - modified_huber: robusta a outliers
- 3 valores de alpha (regularización)
- 3 tipos de tasa de aprendizaje
- 3 valores de tasa de aprendizaje inicial

En total, se prueba 81 combinaciones de hiperparámetros. 

In [46]:
from sklearn.model_selection import GridSearchCV
# ========= Preprocesamiento =========

numeric_features = X.columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features)
])

# ========= Pipeline base =========

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(random_state=42))
])

# ========= Definir hiperparámetros a probar =========

param_grid = {
    'classifier__loss': ['log_loss', 'hinge', 'modified_huber'],
    'classifier__alpha': [0.0001, 0.001, 0.01],  # tasa de regularización
    'classifier__learning_rate': ['constant', 'optimal', 'invscaling'],
    'classifier__eta0': [0.001, 0.01, 0.1]  # tasa de aprendizaje inicial
}

# ========= Configurar GridSearchCV =========

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    return_train_score=True
)

# ========= Ejecutar búsqueda sobre X_train / y_train =========

grid_search.fit(X_train, y_train)

# ========= Reportar resultados =========

results = pd.DataFrame(grid_search.cv_results_)
results_summary = results[['params', 'mean_test_score', 'std_test_score']].sort_values(by='mean_test_score', ascending=False)

# Mostrar las 10 mejores combinaciones
print("Mejores combinaciones (top 5):")
print(results_summary.head(5).to_string(index=False))


Mejores combinaciones (top 5):
                                                                                                                              params  mean_test_score  std_test_score
 {'classifier__alpha': 0.01, 'classifier__eta0': 0.01, 'classifier__learning_rate': 'optimal', 'classifier__loss': 'modified_huber'}         0.875942        0.010999
  {'classifier__alpha': 0.01, 'classifier__eta0': 0.1, 'classifier__learning_rate': 'optimal', 'classifier__loss': 'modified_huber'}         0.875942        0.010999
{'classifier__alpha': 0.01, 'classifier__eta0': 0.001, 'classifier__learning_rate': 'optimal', 'classifier__loss': 'modified_huber'}         0.875942        0.010999
         {'classifier__alpha': 0.01, 'classifier__eta0': 0.001, 'classifier__learning_rate': 'optimal', 'classifier__loss': 'hinge'}         0.875259        0.010590
           {'classifier__alpha': 0.01, 'classifier__eta0': 0.1, 'classifier__learning_rate': 'optimal', 'classifier__loss': 'hinge'}       

In [47]:
# imprimir hiperparámetros del mejor modelo
print(grid_search.best_params_)

{'classifier__alpha': 0.01, 'classifier__eta0': 0.001, 'classifier__learning_rate': 'optimal', 'classifier__loss': 'modified_huber'}


El análisis de validación cruzada arroja que el **mejor modelo es una regresión logística con tasa de aprendizaje constante de 0.01 y un alpha de 0.001**, obteniéndose una accuracy promedio de 87%. 

#### Entrenamiento del mejor modelo

In [48]:
# Extraer el mejor modelo (pipeline completo ya entrenado con los mejores hiperparámetros)
best_model = grid_search.best_estimator_

In [49]:
# Hacer predicciones sobre test
y_test_pred = best_model.predict(X_test)

# Evaluar el modelo
print("\n📊 Evaluación en conjunto de TEST (modelo final)")
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("Precision:", precision_score(y_test, y_test_pred))
print("Recall:", recall_score(y_test, y_test_pred))
print("F1 Score:", f1_score(y_test, y_test_pred))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_test_pred))



📊 Evaluación en conjunto de TEST (modelo final)
Accuracy: 0.8544474393530997
Precision: 0.8
Recall: 0.24242424242424243
F1 Score: 0.37209302325581395
Matriz de confusión:
 [[301   4]
 [ 50  16]]


In [50]:
y_train_pred = best_model.predict(X_train)

print("\n📊 Evaluación en conjunto de ENTRENAMIENTO (modelo final)")
print("Accuracy:", accuracy_score(y_train, y_train_pred))
print("Precision:", precision_score(y_train, y_train_pred))
print("Recall:", recall_score(y_train, y_train_pred))
print("F1 Score:", f1_score(y_train, y_train_pred))
print("Matriz de confusión:\n", confusion_matrix(y_train, y_train_pred))



📊 Evaluación en conjunto de ENTRENAMIENTO (modelo final)
Accuracy: 0.8671611598111936
Precision: 0.8709677419354839
Recall: 0.2222222222222222
F1 Score: 0.3540983606557377
Matriz de confusión:
 [[1232    8]
 [ 189   54]]


#### Interpretación
No hay mucha diferencia entre training y testing. No hay evidencias de overfitting. 



#### Conclusión y perspectivas futuras
Se trata de un modelo conservador. 

El modelo final es razonablemente bueno si el objetivo principal es minimizar falsos positivos (evitar rechazar a buenos clientes).

Si el objetivo del negocio es detectar la mayor cantidad posible de morosos, este modelo necesita mejorar su recall.

Una opción es bajar el umbral de decisión, lo cual mejoraría la recall (mejor detección de casos de mora) a costo de bajar la precisión (más falsos positivos). La decisión de si seguir ese camino depende del tipo de error que sea prioritario minimizar.  

## Ejercicio 3: Árboles de Decisión

En este ejercicio se entrenarán árboles de decisión para predecir la variable objetivo.

Para ello, deberán utilizar la clase DecisionTreeClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/tree.html
  - https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html

### Ejercicio 3.1: DecisionTreeClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador DecisionTreeClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


### Ejercicio 3.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del DecisionTreeClassifier. Como mínimo, probar diferentes criterios de partición (criterion), profundidad máxima del árbol (max_depth), y cantidad mínima de samples por hoja (min_samples_leaf).

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html